In [1]:
import os
import re
import upload_v2 as v2

In [2]:
dict_refer = v2.dict_refer
comp = v2.comp
len(dict_refer)

70792

In [6]:
files = ["20456681 이수진 19카기 양통"]

for f in files:
    print("-----------------------------------------")
    f_name_items = {"key" :"", "name" :"", "docu" :"", "event" :"", "extra" :"", "date" :"", "pw" :""}
    extra = os.path.splitext(f)[0]
    ext = os.path.splitext(f)[1]

    if v2.no_upload(f) : 
        print("업로드 대상 아님")
        continue

    f_name_items["key"], extra = v2.keyFnc(extra) ############

    f_name_items["pw"], extra = v2.pwFnc(extra)  ###### pw 할당

    debtorName = dict_refer[f_name_items["key"]][2] 
    grtName = dict_refer[f_name_items["key"]][3]
    f_name_items["name"], extra = v2.nameFnc(extra, debtorName, grtName) ############
    print(f'name = {f_name_items["name"]},  extra = {extra}')
    
    f_name_items["date"], extra = v2.dateFnc(extra)############
    print(f'date = {f_name_items["date"]}')

    isEvent, docuEventExtra = v2.eventFnc(extra, testMode= True) ############### docu, event
    if isEvent == 1 : # 정상
        f_name_items["docu"] = docuEventExtra[0] #사건번호>>docu 할당 #############
        f_name_items["event"] = docuEventExtra[1]
        extra = docuEventExtra[2]
        print(f'isEvent = {isEvent},  docu = {f_name_items["docu"]},  event = {f_name_items["event"]},  extra = {extra}')
    else : # 사건번호 없거나 경정/항고 중 setDocu할 거
        extra = docuEventExtra[2]
        if isEvent == 2 :
            f_name_items["event"] = docuEventExtra[1]
        print(f'isEvent = {isEvent},  docu = {f_name_items["docu"]},  event = {f_name_items["event"]},  extra = {extra}')

        isDocu, docuNameExtra = v2.setDocu(extra) ##############################
        if isDocu :
            f_name_items["docu"] = docuNameExtra[0]#비사건번호>>docu 할당 #############
            extra = docuNameExtra[1]
            print(f"isDocu : {isDocu},  docu : {f_name_items['docu']},  extra : {extra}")
        else : # nodocu
            print("setDocu 결과 nodocu")
    
    extra = v2.rmNeedless(extra)   
    extra = v2.rm_s(extra)
    f_name_items["extra"] = extra

    new_f = "_".join(filter(lambda x :bool(x), f_name_items.values())) + ext
    print(f, ">>>", new_f)
    

-----------------------------------------
name = 이해철,  extra = 이수진 19카기 양통
date = 
관할법원 제거후 :  이수진 19카기 양통
사건번호 있음. event = 2019카기, extra = 이수진  양통
else사실상 강제집행에 걸림 [d,e,ex]리턴 ('기타', '2019카기', '이수진  양통')
extra에서 기타를 제거하고 1번을 리턴함 [d,e,ex]=('기타', '2019카기', '이수진  양통')
isEvent = 1,  docu = 기타,  event = 2019카기,  extra = 이수진  양통
20456681 이수진 19카기 양통 >>> 20456681_이해철_기타_2019카기_이수진 양통


In [20]:
# 점에 대하여
p_day6 = re.compile(r'\(?\s?(?<!\d)(20)?(?P<y>[01]\d|2[0-2])[\.\/\-\s](?P<m>0[1-9]|1[0-2])[\.\/\-\s](?P<d>[0-2][1-9]|[1-3]0|31)(?!\d)\s?\)?')
f = "2009.06.09"
if p_day6.search(f) :
    d = p_day6.search(f)
    date = d["y"] + d["m"] + d["d"]
    print(date)

090609


In [16]:
p = re.compile(r"(원인|대출)\s?서류|(입회|가입|카드)\s?신청서|(신용)?\s?대출[약신원승확상거][가-힣]*|약정서|(?=녹취)|통화\s?(내용|내역)|(대출)?\s?원장|마이너스\s?대출")
f = "1234567 홍길동 원인서류 녹취록.pdf"
p.search(f)
p.sub("원인서류", f, count=1)

'1234567 홍길동 원인서류 녹취록.pdf'

In [1]:
stem  = "20460946 김태준 집행권원 매입전(가 가소).pdf"
extra = stem
debtorList = []
def ptrnFnc(ptrn) : # 정규식 함수에서 () 인식을 위한 함수
    ptrn = re.sub("\(", "\\(", ptrn) #search에서 ()인식을 위해
    ptrn = re.sub("\)", "\\)", ptrn)
    return ptrn
print(re.findall("[가-힣a-zA-Z]+|\([가-힣]\)|㈜", debtorName))
for ptrn in re.findall("[가-힣a-zA-Z]+|\([가-힣]\)|㈜", debtorName) : #(주)를 안 썼을 수도 있으니 따로 빼준다.
    print("ptrn--------", ptrn)
    if ptrn == "㈜" :
        m = re.search("\(주\)|㈜", stem)
        print("m", m.group())
        if m :
            debtorList.append(m.group())
            extra = re.sub("망?\s?" + ptrnFnc(m.group()) + r"\s?(의(?=\s))?", " ", extra)
    else : 
        ptrn = ptrnFnc(ptrn)
        p = re.search(ptrn, stem)
        if p :
            print("p", p.group())
            debtorList.append(p.group())
            extra = re.sub("망?\s?" + ptrnFnc(p.group()) + r"\s?(의(?=\s))?", " ", extra)
print(debtorList, extra)

NameError: name 're' is not defined

In [ ]:
"기타1" : re.compile(r"기타|집행문\s?부여|배송[가-힣]+|(채권|양도|양수|매매).*계약|화해(?!권고)|대위변제|분할|분납|상환|감면|(상속)?\s?한정\s?승인|상속\s?포기|지방세|세목별|과세|.*내용증명|출입국사실|.*답변서|.*진술서|.*보정(서|명령)|.*인포케어|보증면탈|자동차직권말소|완제|.*품의서|금전\s?공탁|배분\s?[계산|기일|내역]|[가-힣\s]*(예고|답변|준비)|이의신청"), # 제거가 아니므로 첫 글자만 잘 찾으면 된다.

In [9]:
# depth test
files = ["20407898 박한수 양도통지서.pdf"]
for f in files:
    new_f = f 
    depth1, depth2, depth3 = v2.setDepth(new_f)########### depth ###########

    print(depth1, depth2, depth3)

wrongdocu  


In [11]:
import re
stem = "예상감정료산정서 16가단950 사해행위취소 예고"
p = re.compile(r"[가-힣\s]*예고")
p.search(stem)
# fs = ["20437691 이상래(75) 등기부등본.pdf", "20437691 이상래(75) 법인등기부.pdf", "20437691 이상래(75) 배당표 등본.pdf", "20437691 이상래(75) 결정 등본.pdf"]
# for f in fs : 
#     print(p.sub("aa", f))


<re.Match object; span=(16, 26), match=' 사해행위취소 예고'>

In [6]:
comp = { # search는 앞에서부터 찾으니까 엄격한 것이 앞으로. 단 가압류와 압류처럼 가?압류로 표현할 수 있는 것은 순서 상관이 없음
#"1붙은 건 .*를 앞에 붙여서 아예 맨 앞으로 문서구분을 이동시켜도 괜찮겠다. 어차피 이름 중복되면 제거하니까."
        "개인정보" : re.compile(r"개인정보(?!활용|이용| 활| 이)"),
        "개인정보1" : re.compile(r"신분증|(기초)?수급자?|차상위|(법인|사용)?\s?인감|기본\s?증명서?|(가족|혼인)\s?(관계|증명)|이혼|입양|친양자|졸업|병적"),
        "원인서류" : re.compile(r"(원인|대출)\s?서류|(입회|가입|카드)\s?신청서|(신용)?\s?대출[약신원승확상거][가-힣]*|약정서|녹취록?|통화\s?(내용|내역)|(대출)?\s?원장|마이너스\s?대출"),
        "양도통지서" : re.compile(r"(채권)?\s?(양도|양수)\s?통지서?|(채권)?\s?양도\s?및?\s?양수\s?(통지)?서?\s?|(?<![^가-힣][가-힣])양통|(?<=\d차)\s?(양통|양도통지서?)"), # 세양통신 해결
        "양도통지서1" : re.compile(r"종[적족]\s?(조회)?|(?<!주소|소\s)이력"), #종적조회는 모두 양통만 있더라
        # 사건번호 내부는 이제 신경쓰지 않아도 된다.
        "파산" : re.compile(r"파산|면책|파산.*면책"), #파산에도 배당있음. 강제집행보다 먼저 나와야
        # 연도 다음에 나오는 개회가 아닌 경우, 전방탐색을 통해 '개인회생'의 '회생'이 걸리는 거 방지. count도 하자
        "개인회생" : re.compile(r"개회|(개인)?\s?회생"),
        "신용회복" : re.compile(r"(?<=[가-힣]{3})신복|[\s_]신복|신용\s?회복"), #이름에있는 신복,숫자뒤 신복은 제외. 이름다음에 띄어쓰기 없이 나온 신복은.. 
        "신용회복1" : re.compile(r"(개인)?채무\s?조정안?|.*원상\s?회복|신청인\s?현황"),
        "재산조사" : re.compile(r"재산\s?조사|재산\s?조회"), # 상세문서를 재산조사가 대체하는 게 아님에 유의
        "재산조사1" : re.compile(r"(?<!법인)\s등기|(?<!법인)등기|가압류\s?물건지|.*대장|.*등록\s?원부|.*은행거래|.*입출금"),
        
        "부채증명서" : re.compile(r"부채\s?(잔액)?증명\s?[서원]?(류|발급)?"),
        "신용조회" : re.compile(r"신용\s?조[회사]|신용\s?정보(?!( 활용| 이용|활용|이용|동의))"), # 신용조회가 있는 경우 기관이 앞에 나오건 뒤에 나오건 냅두면 되니 상관없음.
        "신용조회1" : re.compile(r"KCB|NICE|나이스", re.I), # 기관명만 있는 경우 docu만 추가하면 됨. 순서대로 검색하니 여기 왔다는 건 신용조회라는 말이 없다는 뜻

        # 등초본
        "외국인증명" : re.compile(r"\(?\s?외국인\s?증명서?\s?\)?|\(?\s?외국인\s?등록\s?사실\s?증명서?\s?\)?|\(?\s?외국인\s?등록증?\s?\)?|\(?\s?외국인\s?\)?"),
        "등초본" : re.compile(r"\(?\s?등본.{0,3}원?초본\s?\)?|\(?\s?원?초본.{0,3}등본\s?\)?|\(?\s?등[\s.,]?초본?\s?\)?|\(?\s?초[\s.,]?등본?\s?\)?|\(?\s?주민\s?등록\s?등본\s?및?\s?초본\s?\)?|\(?\s?주민\s?등록\s?초본\s?및?\s?등본\s?\)?"), # '등본 및 초본' 때문에 .{0,3}
        # 등기부\s?등본은 조건문으로 처리하자.
        "초본" : re.compile(r"(?<![가-힣])원초본|(?<=원)원초본|(?<![가-힣])\(?\s?원\s?초본\s?\)?|(?<=[가-힣]{3})원초본|\(?\s?(?<!법원)초본\s?\)?|\(?\s?주민\s?등록\s?초본\s?\)?"), # 말소자_초본
        "주민등록정보" : re.compile(r"행자부\s?(전송)?\s?(자료)?|주민\s?등록\s?정보|주소\s?이력|법원초본"),
        # 키워드 추가(대체x)
        "기타" : re.compile(r"기타"),
        "기타1" : re.compile(r"집행문\s?부여|배송[가-힣]+|(채권|양도|양수|매매).*계약|화해(?!권고)|대위변제|분할|분납|상환|감면|(상속)?\s?한정\s?승인|상속\s?포기|지방세|세목별|과세|.*내용증명|출입국사실|.*답변서|.*진술서|.*보정(서|명령)|.*인포케어|보증면탈|자동차직권말소|완제|.*품의서|금전\s?공탁|배분\s?[계산|기일|내역]|[가-힣\s]*(예고|답변|준비)|이의신청"), # 제거가 아니므로 첫 글자만 잘 찾으면 된다.
        #기타 제거 : 부채증명서, 신용조회, 주민등록정보
        # 판결문과 결정문이 여러곳에서 나올 수 있어 뒤로 뺌. 결정과 판결은 더욱 애매해서 제외함
        "집행권원" : re.compile(r"집행\s?권원|집행문(?!부여|이의)|승계\s?(집행|결정)?(문)?|판결문|양수금\s?(판결)?문?|(가단|가합|가소|차전)\s?(결정|판결)?문?"), # count=1이 의미있게 하기 위해 올바른 표현도 넣는다.
        "집행권원1" : re.compile(r"사해행위\s?(취소)?\s?(소송)?|공정증서|지급\s?(명령|결정문?)|이행\s?(권고|결정문?)|화해\s?(권고|결정문?)"),
        "집행권원 재도" : re.compile(r"(?<!\b[가-힣])재도(부여|건)?|\b재도건?|(?<!문)\s부여건?|수통\s?(부여)?|재교부건?|재발급건?"),    # 사해행위는 집행권원이지만, 답변서, 예고, 준비서면등이 있어서 키워드로 등록하기가..카단이랑도 엮인경우동 있고..
        "강제집행" : re.compile(r"강제\s?집행|압추|(채권)?\s?압류\s?및?\s?추심\s?(명령|결정)?문?|(채권)?\s?추심\s?및?\s?압류\s?(명령|결정)?문?|채권\s?(추심|압류)\s?(결정)?문?|(카단|카명|카합|카담|타채|타경|타기)\s?(결정)?문?"), # 결정이라는 말이 여러곳에서 나올 수 있어 단독사용은 불가
        "강제집행1" : re.compile(r"재산\s?명시|[가-힣\b]*(부?동산)?\s?(가?압류|경매)(?!물건)|[가-힣\b]*동산|[가-힣\b]*추심|[가-힣\b]*유체|[가-힣\b]*배당[가-힣]|.*진술\s?최고서?"),#일반진술서는 안올려도 되는 거. 올린다면 기타로
        "등본" : re.compile(r"\(?\s?법인\s?등기부?\s?(등본)?\s?\)?|\(?\s?(?<!(등기부|기부 |배당표|당표 |.결정|결정 ))등본\s?\)?|\(?\s?주민\s?등록\s?등본\s?\)?") #법인등본, 제적등본 setDocu에서 따로 처리, 등본이 법원서류에서 자꾸 나오니 젤 아래로...
    }

def setDocu(noKeyStem:str)->list :
    """[False:다큐없음, stem]|[True:다큐있음, [docu, name+"#"+extra]"""
    extra = noKeyStem
    docu = ""
    isDocu = False

    for k, v in comp.items() : #comp의 순서가 의미 있음!!!!!
        if v.search(extra) :
            isDocu = True
            # 문서구분 추가(ss)
            if k == "개인정보1" or k == "양도통지서1" or k == "신용회복1" or k == "재산조사1" or k == "강제집행1" or k == "신용조회1" or k == "기타1" or k=="집행권원1": 
                print("-1자리", k)
                docu = k[ :-1 ] # 1떼기
            # 문서구분이 검색어를 대체
            elif k == "등본" :
                print("등본")
                if re.search("법인", extra) :
                    docu = "법인등기"
                elif re.search("제적", extra) :
                    docu = "제적등본"
                else :
                    docu = k
                extra = v.sub("", extra)
            else :
                docu = k
                extra = v.sub("", extra)
                if k == "집행권원" : # 집행권원 + (재도)? // 재도만 있는 경우는 상위로직 그대로 적용
                    if comp["집행권원 재도"].search(extra):
                        docu = docu + " 재도"
                        extra = comp["집행권원 재도"].sub("", extra)
                elif k == "양도통지서" : # 차수가 있다면 docu로 살려주고, 종적조회는 extra로 넘겨서 살려준다.
                    p_차수 = re.compile(r"(?<!\d)\d차")
                    if p_차수.search(extra) :
                        docu = k + " " + p_차수.search(extra).group()
                        extra = p_차수.sub("", extra)
                    if comp["양도통지서1"].search(extra) : # 종족조회 및 기타 제거 
                        extra = "종적조회"+ " "+ comp["양도통지서1"].sub("", extra)

            extra = re.sub('기타', "", extra, count=1)
            return [isDocu, [docu, extra]] #첫번째로 검색된 곳에서 반복문 종료

    return [isDocu, noKeyStem] #반복문 끝났는데도 매칭되는게 없었다면

In [17]:
def ptrnFnc(ptrn) : # 정규식 함수에서 () 인식을 위한 함수
    ptrn = re.sub("\(", "\\(", ptrn) #search에서 ()인식을 위해
    ptrn = re.sub("\)", "\\)", ptrn)
    return ptrn
stem = "20414390 (주)에스티티에스,민경훈 2020차전108145.pdf"
debtorList = []
debtorName = "㈜에스티티에스"
for ptrn in re.findall("[가-힣a-zA-Z]+|\([가-힣]\)|㈜", debtorName) : #(주)를 안 썼을 수도 있으니 따로 빼준다.
    print("ptrn",ptrn)
    if ptrn == "㈜" :
        m = re.search("\(주\)|㈜", stem)
        if m :
            debtorList.append(m.group())
            print("m",ptrn)
            extra = re.sub(m.group(), "", extra)
    ptrn = ptrnFnc(ptrn)
    p = re.search(ptrn, stem)
    if p :
        print("p", p.group())
        debtorList.append(p.group())
        extra = re.sub("망?\s?" + p.group() + r"\s?(의(?=\s))?", " ", extra) # 의를 지우되 보증인이 의로 시작하는 경우 있어 (?=\s)추가, 대신 앞뒤공백 다 제거할 수 있어서 " "로 대체

if len(debtorList) == 1 :
        name = debtorList[0]
elif len(debtorList) > 1 :
    debtor = " ".join(debtorList)
    # 괄호 앞 뒤로 공백 없애기
    debtor = re.sub("\s\(", "(", debtor)
    debtor = re.sub("\)\s", ")", debtor)
    debtor = re.sub("\s?㈜\s?", "㈜", debtor)
    name = debtor
print(name)

ptrn ㈜
m ㈜
ptrn 에스티티에스
p 에스티티에스
(주)에스티티에스


In [49]:
import pickle

path = r'\\192.168.0.75\스캔파일\스캔파일log\_project\파일\중복조사\kcs별 파일정보_솔림헬프.pkl'
with open(path, "rb") as pkl :
    dict = pickle.load(pkl)

total = len(dict)
print(total)

127903


In [50]:
dict["20414305F16350592693221"]

{'score': 101113,
 'root': '/volume1/솔림헬프/3.집행권원/DNP-01/20414305',
 'stem': '20414305_김기용_집행권원_2014가소5063343__(2)',
 'ext': '.pdf',
 'fullPath': '/volume1/솔림헬프/3.집행권원/DNP-01/20414305/20414305_김기용_집행권원_2014가소5063343__(2).pdf'}

In [31]:
# 중복 그룹 갯수
group = 0
for k, v in dict.items() :
    if len(v) > 5 :
        group += 1
print(group)

0


In [32]:
temp = {}
for key, value in dict.items() :
    c = key[:8]
    k = key[8:16]
    s = key[16:]
    kcs = k + c + s
    temp[kcs] = value
    
len(temp)

127903

In [33]:
temp["20425268F4CE57CC19909"]

{'score': 0,
 'root': '/volume1/스캔파일/새 스캔파일(업로드)',
 'stem': '20425268 신남희 신용회복 원상회복내역 221006',
 'ext': '.pdf',
 'fullPath': '/volume1/스캔파일/새 스캔파일(업로드)/20425268 신남희 신용회복 원상회복내역 221006.pdf'}

In [10]:
dict["F4CE57CC2042526819909"]

[{'score': 0,
  'root': '/volume1/스캔파일/새 스캔파일(업로드)',
  'stem': '20425268 신남희 신용회복 원상회복내역 221006',
  'ext': '.pdf',
  'fullPath': '/volume1/스캔파일/새 스캔파일(업로드)/20425268 신남희 신용회복 원상회복내역 221006.pdf'}]

In [34]:
path = r'\\192.168.0.75\스캔파일\스캔파일log\_project\파일\중복조사\kcs별 파일정보_솔림헬프.pkl'
with open(path, "wb") as pkl :
        pickle.dump(temp, pkl)

In [23]:
hehe = {
    "cks1" : {"fullPath1" : {"score" : "score", "dir" :"dir"}, "fullPath2"  : {"score" : "score", "dir" :"dir"}},
    "cks2" : {"fullPath3" : {"score" : "score", "dir" :"dir"}, "fullPath4"  : {"score" : "score", "dir" :"dir"}},
}
hehe

{'cks1': {'fullPath1': {'score': 'score', 'dir': 'dir'},
  'fullPath2': {'score': 'score', 'dir': 'dir'}},
 'cks2': {'fullPath3': {'score': 'score', 'dir': 'dir'},
  'fullPath4': {'score': 'score', 'dir': 'dir'}}}

In [24]:
hehe["cks1"].update({"fullPath5" : {"score" : "score", "dir" :"dir"}})
hehe

{'cks1': {'fullPath1': {'score': 'score', 'dir': 'dir'},
  'fullPath2': {'score': 'score', 'dir': 'dir'},
  'fullPath5': {'score': 'score', 'dir': 'dir'}},
 'cks2': {'fullPath3': {'score': 'score', 'dir': 'dir'},
  'fullPath4': {'score': 'score', 'dir': 'dir'}}}

In [26]:
hehe["cks1"].keys()

dict_keys(['fullPath1', 'fullPath2', 'fullPath5'])

In [48]:
import os
path = r"\\192.168.0.75\삭제예정파일\중복_서버"
os.chdir(path)
for root, dirs, files in os.walk(path) :
    for f in files :
        src = root
        
        target = root.split("\\")[-1]
        uji = root.replace(target, "")
        new = target[8:16] + target[:8] + target[16:]

        dst = os.path.join(uji, new)

        os.rename(src, dst)
        break
    


In [47]:
import os
path = r"C:\Users\SL\Desktop\test"
os.chdir(path)
for root, dirs, files in os.walk(path) :
    for f in files :
        print(root)
        src = root
        
        target = root.split("/")[-1]
        uji = root.replace(target, "")
        new = target[8:16] + target[:8] + target[16:]

        dst = os.path.join(uji, new)

        os.rename(src, dst)
    


C:\Users\SL\Desktop\test\2314153\kajglafjdkjfagjlh


OSError: [WinError 123] 파일 이름, 디렉터리 이름 또는 볼륨 레이블 구문이 잘못되었습니다: 'C:\\Users\\SL\\Desktop\\test\\2314153\\kajglafjdkjfagjlh' -> '\\SL\\DeskC:\\Userstop\\test\\2314153\\kajglafjdkjfagjlh'